In [ ]:
import numpy as np
import scipy as sp
from scipy import sparse
import matplotlib.pyplot as plt


from modelfcts.backgrounds import (
    update_powerlaw_times_concs, 
    logof10, 
    sample_background_powerlaw,
    sample_ss_conc_powerlaw, 
    decompose_nonorthogonal_basis, 
    update_alternating_inputs, 
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    tags_list_to_csr_matrix, 
    SparseNDArray
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_inverse_transform, 
    truncexp1_density, 
    powerlaw_cutoff_inverse_transform
)
from modelfcts.checktools import check_conc_samples_powerlaw_exp1
from modelfcts.smoothing_function import moving_average
from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition
)
from utils.metrics import jaccard, l2_norm
from modelfcts.ibcm import relu_inplace

# Static $\vec{m}$ equal to background odors (impossible)
To understand if the problem comes from $W$ or $M$. 

In [ ]:
def integrate_inhib_static_m(m_vecs, update_bk, bk_init, inhib_params, bk_params, tmax, dt,
                         seed=None, noisetype="normal", skp=1, activ_fct=None):
    """
    Args:
        m_vecs(np.ndarray): 2d array, shape (number neurons, number dimensions)
            The M matrix where each row is an m vector, ie
            synaptic weights connecting one inhibitory neuron to input neurons.
        update_bk (callable): function that updates the background variables and
            the background vector
        bk_init (list of two 1d np.ndarrays): [bk_vari_init, bk_vec_init]
            bk_vari_init (np.ndarray): array of background random variables,
                shaped [odor, n_var_per_odor], or 1d if one var. per odor.
            bk_vec_init (np.ndarray): initial background vector, must have size n_orn
        inhib_params (list): alpha, beta: list of parameters for the inhibitory
            neurons update. Should have alpha > beta here.
            For the early times averaging of synaptic weights, will use beta
            and alpha as alpha and beta (to keep m vector close to origin).
        bk_params (list): list of parameters passed to update_bk (3rd argument)
        tmax (float): max time
        dt (float): time step
        seed (int): seed for the random number generator
        noisetype (str): either "normal" or "uniform"
        skp (int): save only every skp time step

    Returns:
        tseries, bk_series, bkvec_series, m_series, cbar_series, w_series, s_series
    """
    n_neu = m_vecs.shape[0]  # Number of neurons
    n_orn = m_vecs.shape[1]
    bk_vari_init, bk_vec_init = bk_init
    assert n_orn == bk_vec_init.shape[0], "Mismatch between dimension of m and background"
    alpha, beta = inhib_params

    rng = np.random.default_rng(seed=seed)
    tseries = np.arange(0, tmax, dt*skp)

    # Containers for the solution over time
    bk_series = np.zeros([tseries.shape[0]] + list(bk_vari_init.shape))
    cbar_series = np.zeros([tseries.shape[0], n_neu])
    w_series = np.zeros([tseries.shape[0], n_orn, n_neu])  # Inhibitory weights
    bkvec_series = np.zeros([tseries.shape[0], n_orn])  # Input vecs, convenient to compute inhibited output
    s_series = np.zeros([tseries.shape[0], n_orn])

    ## Initialize running variables, separate from the containers above to avoid side effects.
    cbar = np.zeros(n_neu)  # neuron activities
    wmat = w_series[0].copy()  # Initialize with null inhibition
    bk_vari = bk_vari_init.copy()
    bkvec = bk_vec_init.copy()
    m = m_vecs.copy()
    if activ_fct is None:
        svec = bk_vec_init.copy()
    elif activ_fct == "ReLU":
        svec = relu_inplace(bk_vec_init.copy())
    else: raise ValueError("Unknown activation function: {}".format(activ_fct))

    # Initialize neuron activity with m and background at time zero
    cbar = m.dot(bkvec)

    # Store back some initial values in containers
    cbar_series[0] = cbar
    bk_series[0] = bk_vari
    bkvec_series[0] = bkvec
    s_series[0] = svec

    # Generate N(0, 1) noise samples in advance
    if (tseries.shape[0]*skp-1)*bk_vari.size > 1e7:
        raise ValueError("Too much memory needed; consider calling multiple times for shorter times")
    if noisetype == "normal":
        noises = rng.normal(0, 1, size=(tseries.shape[0]*skp-1,*bk_vari.shape))
    elif noisetype == "uniform":
        noises = rng.random(size=(tseries.shape[0]*skp-1, *bk_vari.shape))
    else:
        raise NotImplementedError("Noise option {} not implemented".format(noisetype))

    t = 0
    for k in range(0, len(tseries)*skp-1):
        t += dt
        ### Inhibitory  weights
        # They depend on cbar and svec at time step k, which are still in cbar, svec
        # cbar, shape [n_neu], should broadcast against columns of wmat,
        # while svec, shape [n_orn], should broadcast across rows (copied on each column)
        wmat = wmat + dt * (alpha*cbar[np.newaxis, :]*svec[:, np.newaxis] - beta*wmat)

        # Update background to time k+1, to be used in next time step
        bkvec, bk_vari = update_bk(bk_vari, bk_params, noises[k], dt)

        # Then, compute activity of IBCM neurons at next time step, k+1,
        # with the updated background and synaptic weight vector m
        # Compute un-inhibited activity of each neuron with current input (at time k)
        cbar = m.dot(bkvec)
        # Lastly, projection neurons at time step k+1
        if activ_fct is None:
            svec = bkvec - wmat.dot(cbar)
        else:
            svec = relu_inplace(bkvec - wmat.dot(cbar))

        # Save current state only if at a multiple of skp
        if (k % skp) == (skp - 1):
            knext = (k+1) // skp
            w_series[knext] = wmat
            bk_series[knext] = bk_vari
            bkvec_series[knext] = bkvec
            cbar_series[knext] = cbar  # Save activity of neurons at time k+1
            s_series[knext] = svec

    return tseries, bk_series, bkvec_series, cbar_series, w_series, s_series

## Attempt with ReLU

In [ ]:
### General simulation parameters
n_dimensions = 25  # Half the real number for faster simulations
n_components = 6
n_neurons = n_components
n_kenyon = int(2000 / 50 * n_dimensions)

# Simulation rates and coupling stay the same (try at least)
duration = 320000.0
deltat = 1.0
inhib_rates = [2.5e-4, 5e-5]  # alpha, beta
# Background components need to be redefined. Extra dimensions are somewhat superfluous

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x21e0b502e992cf60b2b7ea5c9a3ebc46)
back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)
    
# Turbulent background parameters: same rates and constants for all odors
back_params = [
    np.asarray([1.0] * n_components),        # whiff_tmins
    np.asarray([100.] * n_components),       # whiff_tmaxs
    np.asarray([2.0] * n_components),        # blank_tmins
    np.asarray([200.0] * n_components),      # blank_tmaxs
    np.asarray([0.6] * n_components),        # c0s
    np.asarray([0.5] * n_components),        # alphas
]
back_params.append(back_components)

# Initial values of background process variables (t, c for each variable)
init_concs = sample_ss_conc_powerlaw(*back_params[:-1], size=1, rgen=rgen_meta)
init_times = powerlaw_cutoff_inverse_transform(
                rgen_meta.random(size=n_components), *back_params[2:4])
tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

# Initial background vector 
init_bkvec = tc_init[:, 1].dot(back_components)
# nus are first in the list of initial background params
init_back_list = [tc_init, init_bkvec]

# Set m vectors equal to background vectors
constant_m_basis = back_components

# For odor tagging
proj_mat = create_sparse_proj_mat(n_kenyon, n_dimensions, rgen_meta, fraction_filled=6/50)

In [ ]:
# m_init, update_bk, bk_init, ibcm_params, inhib_params, bk_params, tmax, dt, seed=None, noisetype="normal"
skp = 10
sim_results = integrate_inhib_static_m(constant_m_basis, update_powerlaw_times_concs, 
                init_back_list, inhib_rates, back_params, duration, deltat, 
                seed=seed_from_gen(rgen_meta), noisetype="uniform", skp=skp, activ_fct="ReLU")
tser, nuser, bkvecser, cbarser, wser, sser = sim_results

In [ ]:
# Plotting the time course of the dot products -- not interesting with gaussian degeneracy
# Unclear what it shows. 
fig, axes = plot_w_matrix(tser, wser, skp=20, lw=1.5)
pseudo_inv = np.linalg.pinv(constant_m_basis)
for j in range(n_components):
    axes.flat[j].set_ylim(axes.flat[j].get_ylim())
    for i in range(n_dimensions):
        axes.flat[j].axhline(pseudo_inv[i, j], color="grey", ls="--")
        
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(tser, bkvecser, sser, skp=10)

# Compute noise reduction factor, annotate
transient = 150000 // skp
avg_bknorm = np.mean(bknorm_ser[transient:])
avg_snorm = np.mean(snorm_ser[transient:])
avg_reduction_factor = avg_snorm / avg_bknorm
std_bknorm = np.std(bknorm_ser[transient:])
std_snorm = np.std(snorm_ser[transient:])
std_reduction_factor = std_snorm / std_bknorm

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

### Background tag silencing and new odor detection

In [ ]:
def respond_to_x(xvec, mmat, wmat, activ_fct=None):
    # xvec has shape [..., n_orn]
    cvec = xvec.dot(mmat.T)  # cvec has shape [..., n_i]
    svec = xvec - cvec.dot(wmat.T)  # svec has shape [..., n_orn]
    if activ_fct == "ReLU":
        svec = relu_inplace(svec)
    return svec

In [ ]:
def flattened_dict_vals(d, dim1, dim2):
    vals = []
    for i in range(dim1):
        for j in range(dim2):
            vals.append(d[(i, j)])
    return vals

In [ ]:
def tags_list_to_csr_matrix(tag_list, n_neu):
    """ Given a list of sets of active neurons (i.e. a list of tags), 
    and the size of the layer, save to a scipy.sparse.csr_matrix
    where each row is one tag. This minimizes memory use compared
    to saving to a dense padded 2d array, yet is easier to save
    to disk (without pickling) than a list of Python sets. 
    
    Args:
        tag_list (list of sets of ints): a list of sets of
            indices of active neurons
        n_neu (int): number of neurons in the layer, e.g.
            largest possible neuron index + 1. 
    Returns:
        tags_matrix (sp.sparse.csr_matrix): each row is one tag
    """
    tag_lengths = list(map(len, tag_list))
    tags_indptr = np.concatenate([[0], np.cumsum(tag_lengths)])
    tags_indices = np.concatenate(list(map(list, tag_list)), axis=0)
    tags_data = np.ones(len(tags_indices), dtype=bool)
    tags_matrix = sp.sparse.csr_matrix((tags_data, tags_indices, tags_indptr), 
                                   shape=[len(tag_list), n_neu])
    return tags_matrix

def plot_odor_tags(tag_list, tag_times, n_kc):
    tag_lengths = list(map(len, tag_list))
    tag_vectors = tags_list_to_csr_matrix(tag_list, n_kc)
    
    fig, axes = plt.subplots(1, 2, sharey=True)
    axes = axes.flatten()
    fig.set_size_inches(8, 4)
    axes[0].plot(tag_lengths, tag_times/1000, ls="-", marker="s", 
                 ms=4, color="k", lw=1.0)
    axes[0].set(ylabel="Time (x1000)", xlabel="Background tag length")
    axes[0].invert_yaxis()
    barprops = dict(aspect='auto', interpolation='nearest')
    # extent: (left, right, bottom, top)
    axes[1].imshow(tag_vectors.toarray(), **barprops, cmap="binary", 
                   extent=(0, n_kc, tag_times[-1]/1000, tag_times[0]/1000))
    axes[1].set(xlabel="Kenyon cell index")
    fig.tight_layout()
    return fig, axes

In [ ]:
# Check that background tags are silenced
tag_slice = slice(0, int(duration) // skp, 3000 // skp)
tag_slice_range = range(0, int(duration) // skp, 3000 // skp)
tag_times = tser[tag_slice]
back_tags = []
for i in tag_slice_range:
    back_tags.append(project_neural_tag(sser[i], bkvecser[i], proj_mat, 
                        kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=3, fix_thresh=None))

fig, axes = plot_odor_tags(back_tags, tag_times, n_kenyon)
plt.show()
plt.close()

In [ ]:
# Check that new odors make it through still
n_new_odors = 100
test_slice = slice(200000 // skp, int(duration) // skp, 10000 // skp)
test_slice_range = range(200000 // skp, int(duration) // skp, 10000 // skp)
test_slice_times = tser[test_slice]
new_odors = generate_odorant([n_new_odors, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors).reshape(*new_odors.shape[:-1], 1)
mix_svecs = np.zeros([n_new_odors, len(test_slice_times), n_dimensions])
mix_xvecs = np.zeros([n_new_odors, len(test_slice_times), n_dimensions])
#mix_tags = SparseNDArray((n_new_odors, len(test_slice_times), n_kenyon_tag), dtype=bool)
mix_tags = {(i, j):{} for i in range(n_new_odors) for j in range(len(test_slice_times))}
mix_jaccards = np.zeros([n_new_odors, len(test_slice_times)])
new_odors_tags = []

In [ ]:
mix_frac = 0.5
projection_kwargs = dict(kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=3, fix_thresh=None)
for i in range(n_new_odors):
    mix_xvecs[i] = mix_frac*new_odors[i] + bkvecser[test_slice]
    new_odors_tags.append(project_neural_tag(new_odors[i], new_odors[i], proj_mat, **projection_kwargs))
for j, t in enumerate(test_slice_range):
    mix_svecs[:, j] = respond_to_x(mix_xvecs[:, j], constant_m_basis, wser[t], activ_fct="ReLU")
    # Compute all tags now
    for i in range(n_new_odors):
        tag = project_neural_tag(mix_svecs[i, j], mix_svecs[i, j], proj_mat, **projection_kwargs)
        mix_tags[(i, j)] = tag
        mix_jaccards[i, j] = jaccard(new_odors_tags[i], tag)

In [ ]:
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(8, 4)
axes[0].hist(mix_jaccards.flatten(), bins=10)
#axes[1].hist(list(map(len, mix_tags.data)))
mix_tag_lengths = {}
for i in range(n_new_odors):
    for j in range(len(test_slice_times)):
        mix_tag_lengths[(i, j)] = len(mix_tags[(i, j)])

axes[1].scatter(flattened_dict_vals(mix_tag_lengths, n_new_odors, len(test_slice_times)), mix_jaccards.flatten())
plt.show()
plt.close()

## Attempt without ReLU
The learnt W should be closer to the pseudo-inverse, but it turns out not to be the case, really. 

In [ ]:
def main_test(activ_fct="ReLU", )

In [ ]:
# m_init, update_bk, bk_init, ibcm_params, inhib_params, bk_params, tmax, dt, seed=None, noisetype="normal"
skp2 = 10
sim_results = integrate_inhib_static_m(constant_m_basis, update_powerlaw_times_concs, 
                init_back_list, inhib_rates, back_params, duration, deltat, 
                seed=seed_from_gen(rgen_meta), noisetype="uniform", skp=skp2, activ_fct=None)
tser2, nuser2, bkvecser2, cbarser2, wser2, sser2 = sim_results

In [ ]:
fig, ax, bknorm_ser2, snorm_ser2 = plot_background_norm_inhibition(tser2, bkvecser2, sser2, skp=10)

# Compute noise reduction factor, annotate
transient = 150000 // skp
avg_bknorm2 = np.mean(bknorm_ser2[transient:])
avg_snorm2 = np.mean(snorm_ser2[transient:])
avg_reduction_factor2 = avg_snorm2 / avg_bknorm2
std_bknorm2 = np.std(bknorm_ser2[transient:])
std_snorm2 = np.std(snorm_ser2[transient:])
std_reduction_factor2 = std_snorm / std_bknorm2

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

# Check that background tags are silenced
tag_slice = slice(0, int(duration) // skp2, 3000 // skp2)
tag_slice_range = range(0, int(duration) // skp2, 3000 // skp2)
tag_times = tser2[tag_slice]
back_tags = []
for i in tag_slice_range:
    back_tags.append(project_neural_tag(sser2[i], bkvecser2[i], proj_mat, 
                        kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=3, fix_thresh=None))

fig, axes = plot_odor_tags(back_tags, tag_times, n_kenyon)
plt.show()
plt.close()

In [ ]:
# Check that new odors make it through still
n_new_odors = 100
test_slice = slice(200000 // skp2, int(duration) // skp2, 10000 // skp2)
test_slice_range = range(200000 // skp2, int(duration) // skp2, 10000 // skp2)
test_slice_times = tser2[test_slice]
new_odors2 = generate_odorant([n_new_odors, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors2 /= l2_norm(new_odors2).reshape(*new_odors2.shape[:-1], 1)
mix_svecs = np.zeros([n_new_odors, len(test_slice_times), n_dimensions])
mix_xvecs = np.zeros([n_new_odors, len(test_slice_times), n_dimensions])
#mix_tags = SparseNDArray((n_new_odors, len(test_slice_times), n_kenyon), dtype=bool)
mix_tags = {(i, j):{} for i in range(n_new_odors) for j in range(len(test_slice_times))}
mix_jaccards = np.zeros([n_new_odors, len(test_slice_times)])
new_odors_tags2 = []

mix_frac = 0.5
projection_kwargs = dict(kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=3, fix_thresh=None)
for i in range(n_new_odors):
    mix_xvecs[i] = mix_frac*new_odors2[i] + bkvecser2[test_slice]
    new_odors_tags2.append(project_neural_tag(new_odors2[i], new_odors2[i], proj_mat, **projection_kwargs))
for j, t in enumerate(test_slice_range):
    mix_svecs[:, j] = respond_to_x(mix_xvecs[:, j], constant_m_basis, wser2[t], activ_fct=None)
    # Compute all tags now
    for i in range(n_new_odors):
        tag = project_neural_tag(mix_svecs[i, j], mix_svecs[i, j], proj_mat, **projection_kwargs)
        mix_tags[(i, j)] = tag
        mix_jaccards[i, j] = jaccard(new_odors_tags2[i], tag)
        
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(8, 4)
axes[0].hist(mix_jaccards.flatten(), bins=10)
#axes[1].hist(list(map(len, mix_tags.data)))
mix_tag_lengths = {}
for i in range(n_new_odors):
    for j in range(len(test_slice_times)):
        mix_tag_lengths[(i, j)] = len(mix_tags[(i, j)])

axes[1].scatter(flattened_dict_vals(mix_tag_lengths, n_new_odors, len(test_slice_times)), mix_jaccards.flatten())
plt.show()
plt.close()

## Attempt without $W$ decay
The regularization term is what prevents W from being $M^+$. If we do not use ReLU and set $\beta = 0$, then $W$ converges to an approximate pseudo-inverse. 

In [ ]:
inhib_rates3 = [1e-3, 0.0]  # alpha, beta